In [1]:
!mkdir -p ~/agave

%cd ~/agave

!pip3 install --upgrade setvar

import re
import os
import sys
from setvar import *
from time import sleep

# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

/home/jovyan/agave
  Running setup.py bdist_wheel for setvar ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/04/bd/56/49852fe6eb8f012cad3d78579c170a9f76f722e32e78a6f66e
Successfully built setvar
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
writefile("input.txt","""
!INPUT FILE FOR FUNWAVE_TVD
  ! NOTE: all input parameter are capital sensitive
  ! --------------------TITLE-------------------------------------
  ! title only for log file
TITLE = VESSEL
  ! -------------------HOT START---------------------------------
HOT_START = F
FileNumber_HOTSTART = 1
  ! -------------------PARALLEL INFO-----------------------------
  ! 
  !    PX,PY - processor numbers in X and Y
  !    NOTE: make sure consistency with mpirun -np n (px*py)
  !    
PX = 2
PY = 1
  ! --------------------DEPTH-------------------------------------
  ! Depth types, DEPTH_TYPE=DATA: from depth file
  !              DEPTH_TYPE=FLAT: idealized flat, need depth_flat
  !              DEPTH_TYPE=SLOPE: idealized slope, 
  !                                 need slope,SLP starting point, Xslp
  !                                 and depth_flat
DEPTH_TYPE = FLAT
DEPTH_FLAT = 10.0
  ! -------------------PRINT---------------------------------
  ! PRINT*,
  ! result folder
RESULT_FOLDER = output/

  ! ------------------DIMENSION-----------------------------
  ! global grid dimension
Mglob = 500
Nglob = 100

  ! ----------------- TIME----------------------------------
  ! time: total computational time/ plot time / screen interval 
  ! all in seconds
TOTAL_TIME = 3.0
PLOT_INTV = 1.0
PLOT_INTV_STATION = 50000.0
SCREEN_INTV = 1.0
HOTSTART_INTV = 360000000000.0

WAVEMAKER = INI_GAU
AMP = 3.0
Xc = 250.0
Yc = 50.0
WID = 20.0

  ! -----------------GRID----------------------------------
  ! if use spherical grid, in decimal degrees
  ! cartesian grid sizes
DX = 1.0
DY = 1.0
  ! ----------------SHIP WAKES ----------------------------
VESSEL_FOLDER = ./
NumVessel = 2
  ! -----------------OUTPUT-----------------------------
ETA = T
U = T
V = T
""")

Writing file `input.txt'


In [3]:
if os.environ.get('USE_TUNNEL') == 'True': 
    # fetch the hostname and port of the reverse tunnel running in the sandbox 
    # so Agave can connect to our local sandbox
    !echo $(ssh -q -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null sandbox 'curl -s  http://localhost:4040/api/tunnels | jq -r '.tunnels[0].public_url'') > ngrok_url.txt  
    !cat ngrok_url.txt | sed 's|^tcp://||'
    !cat ngrok_url.txt | sed 's|^tcp://||' | sed -r 's#(.*):(.*)#\1#' > ngrok_host.txt
    !cat ngrok_url.txt | sed 's|^tcp://||' | sed -r 's#(.*):(.*)#\2#' > ngrok_port.txt

    # set the environment variables otherwise set when running in a training cluster
    os.environ['VM_PORT'] = readfile('ngrok_port.txt').strip()
    os.environ['VM_MACHINE'] = readfile('ngrok_host.txt').strip()
    os.environ['AGAVE_SYSTEM_HOST'] = readfile('ngrok_host.txt').strip()
    os.environ['AGAVE_SYSTEM_PORT'] = readfile('ngrok_port.txt').strip()
    !echo "VM_PORT=$VM_PORT"
    !echo "VM_MACHINE=$VM_MACHINE"
    setvar("VM_IPADDRESS=$(getent hosts ${VM_MACHINE}|cut -d' ' -f1)")

0.tcp.ngrok.io:12394
Reading file `ngrok_port.txt'
Reading file `ngrok_host.txt'
Reading file `ngrok_host.txt'
Reading file `ngrok_port.txt'
VM_PORT=12394
VM_MACHINE=0.tcp.ngrok.io
VM_IPADDRESS=52.15.183.149


In [4]:
!mkdir -p ~/agave

%cd ~/agave

!pip3 install --upgrade setvar

import re
import os
import sys
import json
from setvar import *
from time import sleep

# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
setvar("""
AGAVE_APP_NAME=training-${AGAVE_USERNAME}
AGAVE_STORAGE_SYSTEM_ID=sandbox-storage-${AGAVE_USERNAME}
AGAVE_EXECUTION_SYSTEM_ID=sandbox-exec-${AGAVE_USERNAME}
AGAVE_SYSTEM_SITE_DOMAIN=localdomain
MACHINE_NAME=sandbox
MACHINE_USERNAME=jovyan
AGAVE_STORAGE_HOME_DIR=/home/${MACHINE_USERNAME}
AGAVE_EXECUTION_SYSTEM_SCRATCH_DIR=/home/${MACHINE_USERNAME}/work
AGAVE_EXECUTION_SYSTEM_WORK_DIR=/home/${MACHINE_USERNAME}/work
AGAVE_STORAGE_WORK_DIR=/home/${MACHINE_USERNAME}
AGAVE_APP_DEPLOYMENT_PATH=agave-deploy
AGAVE_TENANTS_API_BASEURL=https://sandbox.agaveplatform.org/tenants
""")

/home/jovyan/agave
Requirement already up-to-date: setvar in /opt/conda/lib/python3.6/site-packages
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
AGAVE_APP_NAME=training-sampedro
AGAVE_STORAGE_SYSTEM_ID=sandbox-storage-sampedro
AGAVE_EXECUTION_SYSTEM_ID=sandbox-exec-sampedro
AGAVE_SYSTEM_SITE_DOMAIN=localdomain
MACHINE_NAME=sandbox
MACHINE_USERNAME=jovyan
AGAVE_STORAGE_HOME_DIR=/home/jovyan
AGAVE_EXECUTION_SYSTEM_SCRATCH_DIR=/home/jovyan/work
AGAVE_EXECUTION_SYSTEM_WORK_DIR=/home/jovyan/work
AGAVE_STORAGE_WORK_DIR=/home/jovyan
AGAVE_APP_DEPLOYMENT_PATH=agave-deploy
AGAVE_TENANTS_API_BASEURL=https://sandbox.agaveplatform.org/tenants


In [5]:
!tenants-list
!tenants-init -t sandbox
!clients-delete -u "$AGAVE_USERNAME" -p "$AGAVE_PASSWORD" $AGAVE_APP_NAME
!clients-create -u "$AGAVE_USERNAME" -p "$AGAVE_PASSWORD" -N "$AGAVE_APP_NAME" -S
!auth-tokens-create -u $AGAVE_USERNAME -p "$AGAVE_PASSWORD"

sandbox
You are now configured to interact with the APIs at https://sandbox.agaveplatform.org/
Successfully deleted client training-sampedro
Successfully created client training-sampedro
key: Nvdt7rUxNImdEwhygx6s2dRGOWQa 
secret: wAo7fjv_srLbtrUdl0fq_bG0GIUa
Token for sandbox:sampedro successfully refreshed and cached for 14400 seconds
258684f97a46ee22a755dc511dd3958f


In [6]:
if os.environ.get('USE_TUNNEL') == 'True': 
    # fetch the hostname and port of the reverse tunnel running in the sandbox 
    # so Agave can connect to our local sandbox
    !echo $(ssh -q -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null sandbox 'curl -s  http://localhost:4040/api/tunnels | jq -r '.tunnels[0].public_url'') > ngrok_url.txt  
    !cat ngrok_url.txt | sed 's|^tcp://||'
    !cat ngrok_url.txt | sed 's|^tcp://||' | sed -r 's#(.*):(.*)#\1#' > ngrok_host.txt
    !cat ngrok_url.txt | sed 's|^tcp://||' | sed -r 's#(.*):(.*)#\2#' > ngrok_port.txt

    # set the environment variables otherwise set when running in a training cluster
    os.environ['VM_PORT'] = readfile('ngrok_port.txt').strip()
    os.environ['VM_MACHINE'] = readfile('ngrok_host.txt').strip()
    setvar("""
    AGAVE_SYSTEM_HOST=$VM_MACHINE
    AGAVE_SYSTEM_PORT=$VM_PORT
    VM_IPADDRESS=$(getent hosts ${VM_MACHINE}|cut -d' ' -f1)
    """)

0.tcp.ngrok.io:12394
Reading file `ngrok_port.txt'
Reading file `ngrok_host.txt'
AGAVE_SYSTEM_HOST=0.tcp.ngrok.io
AGAVE_SYSTEM_PORT=12394
VM_IPADDRESS=52.15.183.149


In [7]:
!jsonpki --public ~/.ssh/id_rsa.pub > ./id_rsa.pub.txt
!jsonpki --private ~/.ssh/id_rsa > ./id_rsa.txt

In [8]:
os.environ["PUB_KEY"]=readfile("id_rsa.pub.txt").strip()
os.environ["PRIV_KEY"]=readfile("id_rsa.txt").strip()

Reading file `id_rsa.pub.txt'
Reading file `id_rsa.txt'


In [9]:
writefile("${AGAVE_STORAGE_SYSTEM_ID}.txt","""{
    "id": "${AGAVE_STORAGE_SYSTEM_ID}",
    "name": "${MACHINE_NAME} storage (${MACHINE_USERNAME})",
    "description": "The ${MACHINE_NAME} computer",
    "site": "${AGAVE_SYSTEM_SITE_DOMAIN}",
    "type": "STORAGE",
    "storage": {
        "host": "${AGAVE_SYSTEM_HOST}",
        "port": ${AGAVE_SYSTEM_PORT},
        "protocol": "SFTP",
        "rootDir": "/",
        "homeDir": "${AGAVE_STORAGE_HOME_DIR}",
        "auth": {
          "username" : "${MACHINE_USERNAME}",
          "publicKey" : "${PUB_KEY}",
          "privateKey" : "${PRIV_KEY}",
          "type" : "SSHKEYS"
        }
    }
}
""")

Writing file `sandbox-storage-sampedro.txt'


In [10]:
!systems-addupdate -F ${AGAVE_STORAGE_SYSTEM_ID}.txt
!files-list -S ${AGAVE_STORAGE_SYSTEM_ID} ./ | head -5

Successfully added system sandbox-storage-sampedro
/home/jovyan
/home/jovyan/.bash_logout
/home/jovyan/.bashrc
/home/jovyan/.cache
/home/jovyan/.ngrok2


In [11]:
# Edit any parts of this file that you know need to be changed for your machine.
writefile("${AGAVE_EXECUTION_SYSTEM_ID}.txt","""
{
    "id": "${AGAVE_EXECUTION_SYSTEM_ID}",
    "name": "${MACHINE_NAME} (${MACHINE_USERNAME})",
    "description": "The ${MACHINE_NAME} computer",
    "site": "${AGAVE_SYSTEM_SITE_DOMAIN}",
    "public": false,
    "status": "UP",
    "type": "EXECUTION",
    "executionType": "CLI",
    "scheduler" : "FORK",
    "environment": null,
    "scratchDir" : "${AGAVE_EXECUTION_SYSTEM_SCRATCH_DIR}",
    "workDir": "${AGAVE_EXECUTION_SYSTEM_WORK_DIR}",
    "queues": [
        {
            "name": "none",
            "default": true,
            "maxJobs": 10,
            "maxUserJobs": 10,
            "maxNodes": 6,
            "maxProcessorsPerNode": 6,
            "minProcessorsPerNode": 1,
            "maxRequestedTime": "00:30:00"
        }
    ],
    "login": {
        "auth": {
          "username" : "${MACHINE_USERNAME}",
          "publicKey" : "${PUB_KEY}",
          "privateKey" : "${PRIV_KEY}",
          "type" : "SSHKEYS"
        },
        "host": "${AGAVE_SYSTEM_HOST}",
        "port": ${AGAVE_SYSTEM_PORT},
        "protocol": "SSH"
    },
    "maxSystemJobs": 50,
    "maxSystemJobsPerUser": 50,
    "storage": {
        "host": "${AGAVE_SYSTEM_HOST}",
        "port": ${AGAVE_SYSTEM_PORT},
        "protocol": "SFTP",
        "rootDir": "/",
        "homeDir": "${AGAVE_STORAGE_HOME_DIR}",
        "auth": {
          "username" : "${MACHINE_USERNAME}",
          "publicKey" : "${PUB_KEY}",
          "privateKey" : "${PRIV_KEY}",
          "type" : "SSHKEYS"
        }
    }
}""")

Writing file `sandbox-exec-sampedro.txt'


In [12]:
!systems-addupdate -F ${AGAVE_EXECUTION_SYSTEM_ID}.txt
# Test to see if this worked...
!files-list -S ${AGAVE_EXECUTION_SYSTEM_ID} ./ | head -5

Successfully added system sandbox-exec-sampedro
/home/jovyan
/home/jovyan/.bash_logout
/home/jovyan/.bashrc
/home/jovyan/.cache
/home/jovyan/.ngrok2
